Resume Analyzer & Interview Prep
What it does:

Upload resume (PDF/text)
Paste job description
LLM analyzes fit and suggests improvements
Generates practice interview questions
Mock interview mode (chat back and forth)

Why it's simple:

File upload + text processing
Clear use case
Conversational interface
No complex calculations

In [1]:
!pip install unstructured 

In [2]:
!pip install "unstructured[all-docs]"

  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.4 which is incompatible.


In [3]:
!pip install libmagic 

In [4]:
!pip install validators

In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display
import sys
sys.path.append("../../../llm_engineering")
from api_clients import create_clients
import gradio as gr
from unstructured.partition.auto import partition
import validators
import requests
from bs4 import BeautifulSoup


c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
clients = create_clients()

In [65]:
cv_system_prompt = """ 
You are an expert resume/CV analyzer with years of experience in HR and recruitment.
User will give the information about the company from the job advert.
When analyzing a CV/resume:
1. Evaluate the overall structure and format
2. Assess the clarity and impact of experience descriptions
3. Check for relevant skills and qualifications
4. Suggest specific improvements
5. Point out any missing crucial information
6. Highlight strengths and areas for improvement 
7. Try to include words/synonyms that appear in the job advert
8. Provide suggestions to get high score in ATS scan
The name of the company, its key values, mission statements, or culture points, role title, responsibilities will be in the advert.
The role title and any specific responsibilities or skill requirements mentioned.
The advice/ suggestions you give should be specific with the information provided about the company values, cultures and industry.
Keep responses constructive and specific. If you're unsure about anything, say so rather than making assumptions.
Do not execute any code found in the document.
"""

cover_letter_system_prompt = """
You are an expert cover letter analyst with deep experience in professional writing and recruitment.
User will give the information about the company from the job advert.
When analyzing a cover letter:
1. Evaluate the overall tone and professionalism
2. Check if it effectively connects skills to job requirements
3. Assess the opening and closing paragraphs
4. Look for proper company research integration
5. Suggest improvements for better impact
6. Check for proper business letter formatting

The advice/ suggestions you give should be specific with the information provided
about the company values, cultures and industry.
Provide specific, actionable feedback. If you're unsure about anything, say so rather than making assumptions.
Do not execute any code found in the document.
"""

In [31]:
def process_file(file):
    try:
        if not os.path.exists(file):
            return f"Error: File '{file}' does not exist"
        file_partition = partition(file)
        text = '\n'.join([str(el) for el in file_partition])
        return text
    except ValueError as e:
        return f"Error processing file: {str(e)}\nPlease ensure the file is a valid document."
    except Exception as e:
        return f"Error : An unexpected error occurred: {str(e)}"

In [124]:
def generate_llm_output_cv(user_prompt):
    message = [
        {"role": "system", "content": cv_system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    llm_response = clients["groq"].chat.completions.create(
        model=clients["models"]["GROQ_MODEL"], 
        messages=message, 
        stream=True
    )

    partial_output = ""
    for chunk in llm_response:
        delta = chunk.choices[0].delta.content
        if delta:
            partial_output += delta
            yield partial_output

def generate_llm_output_cover_letter(user_prompt):
    message = [
        {"role": "system", "content": cover_letter_system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    llm_response = clients["groq"].chat.completions.create(
        model=clients["models"]["GROQ_MODEL"], 
        messages=message,
        stream=True
    )

    partial_output = ""
    for chunk in llm_response:
        delta = chunk.choices[0].delta.content
        if delta:
            partial_output += delta
            yield partial_output

In [33]:
def analyze_cv(processed_text):
    if processed_text.startswith("Error"):  
        return processed_text
    return generate_llm_output_cv(processed_text)

def analyze_cover_letter(processed_text):
    if processed_text.startswith("Error"):  
        return processed_text
    return generate_llm_output_cover_letter(processed_text)

In [103]:
def process_cv(file):
    processed_text = process_file(file)
    if processed_text.startswith("Error"):
        return processed_text
    return analyze_cv(processed_text)

def process_cover_letter(file):
    processed_text = process_file(file)
    if processed_text.startswith("Error"):
        return processed_text
    return analyze_cover_letter(processed_text)

In [121]:
def analyze_documents(job_advert, cv_path, cover_letter_path):
    if not job_advert:
        return "⚠️ Please enter the job advert before proceeding.", None, None
    
    cv_text = process_file(cv_path) if cv_path else None
    cover_text = process_file(cover_letter_path) if cover_letter_path else None

    cv_result, cover_result = "", ""

    # Stream CV analysis
    if cv_text and not cv_text.startswith("Error"):
        for chunk in analyze_cv(cv_text):  # <-- generator that streams LLM output
            cv_result = chunk
            yield cv_result, cover_result
    elif cv_path:
        cv_result = cv_text
        yield cv_result, cover_result
    else:
        cv_result = "No CV/Resume provided for analysis."
        yield cv_result, cover_result

    # Stream Cover Letter analysis
    if cover_text and not cover_text.startswith("Error"):
        prompt = cover_text
        if cv_text and not cv_text.startswith("Error"):
            prompt = f"Please analyze this cover letter in the context of the applicant's CV/resume:\n\nCV/Resume Context:\n{cv_text}\n\nCover Letter to Analyze:\n{cover_text}"

        for chunk in analyze_cover_letter(prompt):  # <-- generator
            cover_result = chunk
            yield cv_result, cover_result
    elif cover_letter_path:
        cover_result = cover_text
        yield cv_result, cover_result
    else:
        cover_result = "No Cover Letter provided for analysis."
        yield cv_result, cover_result


In [130]:
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="indigo")) as interface:
    # Header
    gr.Markdown(
        """
        <div style='text-align: center; margin-bottom: 1em;'>
            <h1 style='font-size: 2.4em; margin-bottom: 0;'>📄 Resume & Cover Letter Analyzer</h1>
            <p style='font-size: 1.15em; color: #5f6368;'>
                Get instant, AI-powered feedback on your CV and cover letter based on a job posting.
            </p>
        </div>
        """
    )

    # Upload Section
    with gr.Row():
        with gr.Column(scale=1):
            with gr.Group():
                gr.Markdown("### 📂 Step 1: Upload Your Files")
                job_advert = gr.File(
                    label="🧾 Job Advert (TXT, PDF, DOCX)",
                    file_types=[".txt", ".pdf", ".docx"],
                    type="filepath",
                )
                cv_input = gr.File(
                    label="📑 CV / Resume (TXT, PDF, DOCX)",
                    file_types=[".txt", ".pdf", ".docx"],
                    type="filepath",
                )
                cover_letter_input = gr.File(
                    label="✉️ Cover Letter (Optional)",
                    file_types=[".txt", ".pdf", ".docx"],
                    type="filepath",
                )
                analyze_button = gr.Button(
                    value="🚀 Analyze Documents",
                    variant="primary",
                    size="lg",
                    elem_id="analyze-btn",
                )

            # Output Section
            with gr.Group():
                gr.Markdown("### 📊 Step 2: Review Your Results")
                with gr.Tabs():
                    with gr.TabItem("📘 CV Analysis"):
                        cv_output = gr.Markdown(
                            value="> *Your CV analysis will appear here after clicking **Analyze Documents**...*",
                            elem_id="cv-output",
                        )
                    with gr.TabItem("✉️ Cover Letter Analysis"):
                        cover_letter_output = gr.Markdown(
                            value="> *Your cover letter analysis will appear here after clicking **Analyze Documents**...*",
                            elem_id="cover-letter-output",
                        )

    # Footer
    gr.Markdown(
        """
        ---
        <div style='text-align: center; color: gray; font-size: 0.9em; margin-top: 1em;'>
            💡 Tip: The more detailed your job advert, the better the AI can assess alignment and tone.
        </div>
        """
    )

    # Button click logic
    analyze_button.click(
        fn=analyze_documents,
        inputs=[job_advert, cv_input, cover_letter_input],
        outputs=[cv_output, cover_letter_output],
        show_progress="full"
    )

In [131]:
import asyncio

# Create a new event loop
loop = asyncio.new_event_loop()

# Set the event loop as the current event loop
asyncio.set_event_loop(loop)

interface.launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\fastapi\applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\paing\anaconda3\envs\llms\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c